In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 设定参数
T = 10          # 总时间
N = 500         # 步数
dt = T / N      # 每步时间间隔
np.random.seed(0)  # 保持随机数可复现

# 生成二维Wiener过程
dW = np.sqrt(dt) * np.random.randn(N, 2)  # 每步增量
W = np.cumsum(dW, axis=0)                 # 积分累加轨迹
W = np.vstack((np.zeros((1, 2)), W))       # 加上起点 (0,0)

# 绘制动画
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(W[:, 0].min() - 1, W[:, 0].max() + 1)
ax.set_ylim(W[:, 1].min() - 1, W[:, 1].max() + 1)
ax.set_title("2D Wiener Process Evolution")
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    line.set_data(W[:i, 0], W[:i, 1])
    return line,

ani = animation.FuncAnimation(fig, animate, frames=N+1, init_func=init,
                              interval=20, blit=True)

# 保存动画
ani.save("wiener_process.mp4", writer='ffmpeg', fps=30)
plt.close()


# 二维 Wiener 过程（二维布朗运动）

本实验模拟二维平面上的 Wiener 过程轨迹，并以动画形式展示随时间演化的过程。

## 理论背景

二维 Wiener 过程 $ \mathbf{W}(t) = (W_1(t), W_2(t)) $ 定义为：

- $ W_1(t)$ 和 $W_2(t) $ 是**独立的一维标准 Wiener 过程**。
- 起点满足：
  $$
  \mathbf{W}(0) = (0, 0)
  $$
- 对任意 \( t > s \)，增量服从正态分布：
  $$
  W_i(t) - W_i(s) \sim \mathcal{N}(0, t-s), \quad i = 1,2
  $$

## 数值实现

离散时间步长 $\Delta t$下，二维 Wiener 过程的生成可以表示为：

- 每一步增量：
  $$
  \Delta W_i \sim \mathcal{N}(0, \Delta t)
  $$
- 轨迹通过累加每步增量得到：
  $$
  W_i(t_{n+1}) = W_i(t_n) + \Delta W_i
  $$

In [1]:
from IPython.display import HTML

HTML("""
<video width="500" height="500" controls>
  <source src="wiener_process.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# 参数设置
T = 10           # 总时间
N = 500          # 时间步数
dt = T / N       # 每一步的时间间隔
np.random.seed(0)  # 保持随机数可复现

# 生成三维Wiener过程
dW = np.sqrt(dt) * np.random.randn(N, 3)  # 每步增量
W = np.cumsum(dW, axis=0)                 # 积分得到轨迹
W = np.vstack((np.zeros((1, 3)), W))       # 起点加(0,0,0)

# 创建3D动画
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(W[:,0].min()-1, W[:,0].max()+1)
ax.set_ylim(W[:,1].min()-1, W[:,1].max()+1)
ax.set_zlim(W[:,2].min()-1, W[:,2].max()+1)
ax.set_title("3D Wiener Process Evolution")

line, = ax.plot([], [], [], lw=2)

def init():
    line.set_data([], [])
    line.set_3d_properties([])
    return line,

def animate(i):
    line.set_data(W[:i, 0], W[:i, 1])
    line.set_3d_properties(W[:i, 2])
    return line,

ani = animation.FuncAnimation(fig, animate, frames=N+1, init_func=init,
                              interval=20, blit=True)

# 保存为 mp4
ani.save("wiener_process_3d.mp4", writer='ffmpeg', fps=30)
plt.close()


In [5]:
HTML("""
<video width="600" height="600" controls>
  <source src="wiener_process_3d.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

# 参数设置
T = 10
N = 500
dt = T / N
np.random.seed(0)

# 生成三维 Wiener 过程
dW = np.sqrt(dt) * np.random.randn(N, 3)
W = np.cumsum(dW, axis=0)
W = np.vstack((np.zeros((1, 3)), W))  # 起点(0,0,0)

# 创建线段数据（每一小段轨迹）
points = W.reshape(-1, 1, 3)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# 创建3D图
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(W[:,0].min()-1, W[:,0].max()+1)
ax.set_ylim(W[:,1].min()-1, W[:,1].max()+1)
ax.set_zlim(W[:,2].min()-1, W[:,2].max()+1)
ax.set_title("3D Wiener Process with Color Evolution")

# --- 关键修改: 提供一个很小的初始段 (不是完全空的) ---
init_segments = segments[:1]  # 取前1小段，避免空list

lc = Line3DCollection(init_segments, cmap='viridis', norm=plt.Normalize(0, N))
lc.set_array(np.linspace(0, N, N))  # 映射色条
ax.add_collection3d(lc)

def init():
    lc.set_segments(init_segments)  # 初始至少1段
    return lc,

def animate(i):
    if i == 0:
        lc.set_segments(init_segments)
    else:
        current_segments = segments[:i]
        lc.set_segments(current_segments)
    return lc,

ani = animation.FuncAnimation(fig, animate, frames=N+1, init_func=init,
                              interval=20, blit=True)

# 保存为 mp4
ani.save("wiener_process_3d_colored.mp4", writer='ffmpeg', fps=30)
plt.close()


In [2]:
HTML("""
<video width="600" height="600" controls>
  <source src="wiener_process_3d_colored.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from IPython.display import HTML

# 参数设置
T = 20           # 总时间
N = 500*2          # 步数
dt = T / N
np.random.seed(0)

# 螺旋漂移参数
omega = 2 * np.pi / 5   # 螺旋角频率（控制转一圈需要多久）
v = 0.5                 # z轴向上的漂移速度

# 时间戳
ts = np.linspace(0, T, N)

# 漂移项（沿螺旋线）
mu = np.zeros((N, 3))
mu[:, 0] = np.cos(omega * ts)  # x方向漂移
mu[:, 1] = np.sin(omega * ts)  # y方向漂移
mu[:, 2] = v                   # z方向漂移

# 生成带漂移的随机过程
dW = np.sqrt(dt) * np.random.randn(N, 3) *0.1   # 随机增量
dX = mu * dt + dW                           # 总增量 = 漂移 + 扩散
X = np.cumsum(dX, axis=0)                   # 积分轨迹
X = np.vstack((np.zeros((1, 3)), X))         # 起点 (0,0,0)

# 创建线段数据
points = X.reshape(-1, 1, 3)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# 创建3D图
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(X[:,0].min()-1, X[:,0].max()+1)
ax.set_ylim(X[:,1].min()-1, X[:,1].max()+1)
ax.set_zlim(X[:,2].min()-1, X[:,2].max()+1)
ax.set_title("3D Wiener Process with Spiral Drift")

# 提供初始段
init_segments = segments[:1]

lc = Line3DCollection(init_segments, cmap='viridis', norm=plt.Normalize(0, N))
lc.set_array(np.linspace(0, N, N))  # 时间色条
ax.add_collection3d(lc)

def init():
    lc.set_segments(init_segments)
    return lc,

def animate(i):
    if i == 0:
        lc.set_segments(init_segments)
    else:
        current_segments = segments[:i]
        lc.set_segments(current_segments)
    return lc,

ani = animation.FuncAnimation(fig, animate, frames=N+1, init_func=init,
                              interval=20, blit=True)

# 保存
ani.save("wiener_process_spiral_drift.mp4", writer='ffmpeg', fps=30)
plt.close()


In [3]:
from IPython.display import HTML

HTML("""
<video width="600" height="600" controls>
  <source src="wiener_process_spiral_drift.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation

# 模拟器参数
nx, ny = 128, 128     # 网格大小
dx = dy = 1.0         # 空间步长
dt = 0.01             # 时间步长
T = 200.0              # 总时间
n_steps = int(T / dt)

epsilon = 1.0         # 界面宽度控制参数
noise_strength = 0.05 # 初始扰动强度

# 初始化相场变量 φ（随机扰动围绕0.5）
phi = 0.5 + noise_strength * (np.random.rand(nx, ny) - 0.5)

# 用于存储动画帧
snapshots = [phi.copy()]

# Allen-Cahn PDE 右边项
def double_well_potential_deriv(phi):
    return phi * (phi - 1) * (2 * phi - 1)  # d/dφ [φ²(1-φ)²]

# 拉普拉斯算子（周期边界条件）
def laplacian(phi):
    return (
        -4 * phi
        + np.roll(phi, 1, axis=0)
        + np.roll(phi, -1, axis=0)
        + np.roll(phi, 1, axis=1)
        + np.roll(phi, -1, axis=1)
    ) / (dx * dy)

# 时间演化
for step in range(n_steps):
    deltaF = -epsilon**2 * laplacian(phi) + double_well_potential_deriv(phi)
    phi += -dt * deltaF  # Allen-Cahn更新
    if step % 50 == 0:
        snapshots.append(phi.copy())

# 可视化动画
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(snapshots[0], cmap=cm.viridis, origin='lower', vmin=0, vmax=1)
ax.set_title("Allen-Cahn Phase Field Evolution")
ax.axis('off')

def update(frame):
    im.set_array(snapshots[frame])
    return [im]

ani = FuncAnimation(fig, update, frames=len(snapshots), interval=50)

# 保存到当前目录下
output_path = "allen_cahn_simulation.mp4"
ani.save(output_path, writer='ffmpeg', fps=10)

plt.close()
print(f"动画已保存为 {output_path}")


动画已保存为 allen_cahn_simulation.mp4


In [4]:
from IPython.display import HTML

HTML("""
<video width="500" height="500" controls>
  <source src="allen_cahn_simulation.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")
